In [2]:
import json
import os
import collections.abc
import csv

def get_metrics(res_dir):
  with open(f"{res_dir}/trainlog.csv") as f:
    reader = csv.reader(f)
    keys =next(reader)
    metrics = {k:[] for k in keys}
    
    for row in reader:
      for k,v in zip(keys, row):
        metrics[k].append(float(v))
  return metrics

def get_accs_at_epoch(m, epoch, test_acc=None):
  accs = {}
  for k in ['accuracy', 'val_accuracy', 'test_accuracy']:
    if k in m:
      accs[k] = round(m[k][epoch] * 100, 1)
  if test_acc is not None and 'test_accuracy' not in accs:
    accs['test_accuracy'] = round(test_acc * 100, 1)
  
  return accs

def get_results(week):
  results = []
  for name in os.listdir(f"{week}"):
    if name.startswith("-"):
      continue
    for seed in os.listdir(f"{week}/{name}"):
      if seed.startswith("-"):
        continue
      item = {
        "name": name,
        'week': week,
        'seed': seed[4:]
      }

      
      if os.path.exists(f"{week}/{name}/{seed}/config.json"):
        with open(f"{week}/{name}/{seed}/config.json") as f:
          config = json.load(f)

        item.update(config)
        
      
      if os.path.exists(f"{week}/{name}/{seed}/overview.json"):
        with open(f"{week}/{name}/{seed}/overview.json") as f:
          overview = json.load(f)
        
        test_accuracy = overview.get("test_accuracy", None)
        if isinstance(test_accuracy, collections.abc.Sequence):
          test_accuracy = test_accuracy[0]

        item.update({"test_accuracy": test_accuracy, 'train_time': round(overview.get("train_time", 0) / (60*60), 2)})
      
      if os.path.exists(f"{week}/{name}/{seed}/trainlog.csv"):
        train_log = get_metrics(f"{week}/{name}/{seed}")
        max_val = train_log['val_accuracy'].index(max(train_log['val_accuracy']))
        accs = get_accs_at_epoch(train_log, max_val, item.get('test_accuracy', None))
        item.update(accs)
        item.update({"best_val_epoch": max_val + 1})
      else: 
        continue

      results.append(item)
  return results
       

In [29]:
keys = ['name', 'week', 'seed', 'accuracy', 'val_accuracy',  'test_accuracy', 'best_val_epoch', 'G_epoch', 'hidden_size', 'batch_size', 'learning_rate', 'dropout', 'train_time',]
from tabulate import tabulate
def print_week(week):
  results = get_results(week)
  data = [[r.get(k, '') for k in keys] for r in results]
  #print(data)
  print(tabulate(data, headers=keys))
  #print (tabulate(results, headers=tuple(keys)))#, headers=keys))
  # for r in results:
  #   print("\t".join([str(r.get(k, '')) for k in keys]))

In [30]:

#print("\t".join(keys))
#print_week("week47")
#print_week("week48")
print_week("week49")

name                                 week      seed    accuracy    val_accuracy  test_accuracy      best_val_epoch    G_epoch    hidden_size    batch_size    learning_rate    dropout  train_time
-----------------------------------  ------  ------  ----------  --------------  ---------------  ----------------  ---------  -------------  ------------  ---------------  ---------  ------------
G-LIGHT-GRU128-G1-LR3-B32-REG-DROP   week49      89        98.6            22.1  30.9                          169          1            128            32           0.001         0.5  3.56
G-LIGHT-GRU128-G1-LR7-B32-REG        week49     196        78.6            48                                    1          1            128            32           1e-07         0
G-LIGHT-GRU128-G1-LR7-B32-REG        week49      89        89.6            55.5  63.7                          997          1            128            32           1e-07         0    12.58
G-LIGHT-GRU128-G1-LR4-B256-REG       week49     